In [41]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
def connect_to_database():
    """Function for connecting to the SQL database"""

    # Create variables to store cretentials
    USER = "admin"
    DB = "dbikes"
    PORT = "3306"
    URL = "database-1.ctesjcult8dm.eu-west-1.rds.amazonaws.com"

    # Read in password from text file
    with open('../mysql_password.txt') as f:
        PASSWORD = ''.join(f.readlines())
        PASSWORD = str(PASSWORD).split()[0]

    # Create engine using credentials
    engine = create_engine(
        "mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)

    # Create connection using engine and return connection
    conn = engine.connect()
    return conn


In [3]:
engine = connect_to_database()

2022-03-23 14:08:47,233 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-03-23 14:08:47,235 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 14:08:47,254 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-03-23 14:08:47,255 INFO sqlalchemy.engine.Engine [generated in 0.00659s] {}
2022-03-23 14:08:47,290 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-23 14:08:47,291 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
availability_df = pd.read_sql_query("SELECT * FROM dbikes.availability", engine)
availability_df.head()

2022-03-23 14:08:47,453 INFO sqlalchemy.engine.Engine SELECT * FROM dbikes.availability
2022-03-23 14:08:47,455 INFO sqlalchemy.engine.Engine [raw sql] {}


,number,available_bikes,available_stands,last_update
0,42,17,13,1644921618000
1,30,15,5,1644922002000
2,54,16,17,1644922012000
3,108,10,25,1644921926000
4,56,9,31,1644922071000


In [5]:
availability_df.shape

(2827330, 4)

In [6]:
# Remove duplicates
availability_df.drop_duplicates(inplace=True)

In [7]:
print(availability_df.shape)
# availability_df.duplicated().sum()

(718191, 4)


In [21]:
weather_df = pd.read_sql_query("SELECT dt, temperature, main FROM dbikes.real_time_weather", engine)
weather_df.head()

2022-03-23 14:10:56,343 INFO sqlalchemy.engine.Engine SELECT dt, temperature, main FROM dbikes.real_time_weather
2022-03-23 14:10:56,343 INFO sqlalchemy.engine.Engine [raw sql] {}


,dt,temperature,main
0,1644922267,280,Clouds
1,1644922316,280,Clouds
2,1644922617,280,Clouds
3,1644922917,280,Clouds
4,1644923217,280,Clouds


In [9]:
#Drop duplicates
weather_df.drop_duplicates(inplace=True)

In [10]:
print(weather_df.shape)

(10398, 3)


In [11]:
#Changes the name of the column
availability_df.rename(columns={"last_update": "dt"}, inplace=True)
availability_df.columns

Index(['number', 'available_bikes', 'available_stands', 'dt'], dtype='object')

In [12]:
availability_df["dt"] = availability_df["dt"].astype(np.int64)
availability_df.dtypes

number              int64
available_bikes     int64
available_stands    int64
dt                  int64
dtype: object

In [13]:
weather_df["dt"]= weather_df["dt"].astype(np.int64)
weather_df.dtypes

dt              int64
temperature     int64
main           object
dtype: object

In [14]:
availability_df.sort_values(by="dt", inplace=True)
weather_df.sort_values(by="dt", inplace=True)

In [15]:
#Converts seconds to miliseconds
availability_df["dt"] = availability_df["dt"] // 1000
availability_df.head()

,number,available_bikes,available_stands,dt
86,86,0,38,1644921498
34,55,14,21,1644921501
98,80,22,18,1644921515
7,32,15,15,1644921522
30,99,15,15,1644921535


In [16]:
df=pd.merge_asof(availability_df, weather_df, on="dt", direction="nearest")
df.head(20)

,number,available_bikes,available_stands,dt,temperature,main
0,86,0,38,1644921498,280,Clouds
1,55,14,21,1644921501,280,Clouds
2,80,22,18,1644921515,280,Clouds
3,32,15,15,1644921522,280,Clouds
4,99,15,15,1644921535,280,Clouds
5,115,0,30,1644921543,280,Clouds
6,9,24,0,1644921545,280,Clouds
7,18,18,12,1644921589,280,Clouds
8,39,8,12,1644921597,280,Clouds
9,13,15,15,1644921598,280,Clouds


In [17]:
df.shape

(718191, 6)

In [18]:
df["temperature"] = df["temperature"] - 273.15
df

,number,available_bikes,available_stands,dt,temperature,main
0,86,0,38,1644921498,6.85,Clouds
1,55,14,21,1644921501,6.85,Clouds
2,80,22,18,1644921515,6.85,Clouds
3,32,15,15,1644921522,6.85,Clouds
4,99,15,15,1644921535,6.85,Clouds
...,...,...,...,...,...,...
718186,23,29,1,1648044418,15.85,Clear
718187,47,24,16,1648044423,15.85,Clear
718188,109,1,28,1648044435,15.85,Clear
718189,11,16,14,1648044439,15.85,Clear


In [19]:
df["temperature"].value_counts()

 7.85     125364
 8.85     100116
 6.85      89276
 5.85      75979
 4.85      58264
 3.85      49589
 9.85      46716
 10.85     38398
 2.85      36476
 1.85      24514
 11.85     18931
 12.85     16984
 0.85      15828
-0.15       9559
 14.85      4236
 13.85      4060
 15.85      2936
-1.15        965
Name: temperature, dtype: int64

In [20]:
df["main"].value_counts()

Clouds     605989
Rain        62730
Clear       32832
Drizzle     13784
Snow         1844
Mist         1012
Name: main, dtype: int64

In [23]:
df["dt"] = pd.to_datetime(df["dt"], unit="s")
df["day_of_week"] = df["dt"].dt.day_name()
df.head()


,number,available_bikes,available_stands,dt,temperature,main,day_of_week
0,86,0,38,2022-02-15 10:38:18,6.85,Clouds,Tuesday
1,55,14,21,2022-02-15 10:38:21,6.85,Clouds,Tuesday
2,80,22,18,2022-02-15 10:38:35,6.85,Clouds,Tuesday
3,32,15,15,2022-02-15 10:38:42,6.85,Clouds,Tuesday
4,99,15,15,2022-02-15 10:38:55,6.85,Clouds,Tuesday


In [25]:
df["hour"] = df["dt"].dt.hour
df.head()

,number,available_bikes,available_stands,dt,temperature,main,day_of_week,hour
0,86,0,38,2022-02-15 10:38:18,6.85,Clouds,Tuesday,10
1,55,14,21,2022-02-15 10:38:21,6.85,Clouds,Tuesday,10
2,80,22,18,2022-02-15 10:38:35,6.85,Clouds,Tuesday,10
3,32,15,15,2022-02-15 10:38:42,6.85,Clouds,Tuesday,10
4,99,15,15,2022-02-15 10:38:55,6.85,Clouds,Tuesday,10


In [27]:
df.dtypes

number                       int64
available_bikes              int64
available_stands             int64
dt                  datetime64[ns]
temperature                float64
main                        object
day_of_week                 object
hour                         int64
dtype: object

In [29]:
#Changes the value type to categorical
df["main"] = df["main"].astype("category")
df["day_of_week"] = df["day_of_week"].astype("category")
df["hour"] = df["hour"].astype("category")

In [30]:
df.dtypes

number                       int64
available_bikes              int64
available_stands             int64
dt                  datetime64[ns]
temperature                float64
main                      category
day_of_week               category
hour                      category
dtype: object

In [24]:
df["day_of_week"].value_counts()

Tuesday      116242
Wednesday    113274
Thursday     101886
Monday       101637
Friday        99867
Saturday      93964
Sunday        91321
Name: day_of_week, dtype: int64

In [37]:
station_numbers = list(df["number"].unique())
print(station_numbers)

[86, 55, 80, 32, 99, 115, 9, 18, 39, 13, 62, 117, 12, 42, 82, 107, 28, 65, 40, 95, 76, 47, 114, 15, 75, 24, 116, 25, 26, 16, 102, 59, 57, 112, 64, 3, 106, 109, 101, 21, 11, 60, 58, 93, 63, 5, 111, 2, 87, 17, 27, 44, 91, 73, 103, 51, 37, 7, 110, 97, 61, 89, 69, 38, 85, 52, 72, 104, 108, 98, 84, 78, 31, 68, 66, 94, 45, 36, 41, 50, 43, 48, 71, 105, 19, 96, 34, 30, 6, 22, 100, 10, 67, 54, 83, 23, 4, 8, 90, 88, 113, 77, 74, 29, 56, 79, 53, 49, 92, 33]


In [60]:
model={}

for station in station_numbers:
    station_df = df[df["number"] == station]
    X = station_df[["main", "hour", "day_of_week", "temperature"]]
    y = station_df["available_bikes"]
    dummy_fields = ["main", "hour", "day_of_week"]
    for each in dummy_fields:
        dummies = pd.get_dummies(X[each], prefix=each, drop_first =False)
        X = pd.concat([X, dummies], axis=1)
    X = X.drop(dummy_fields, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model[station] = LogisticRegression( solver='lbfgs', max_iter=1500)
    test = model[station].fit(X_train, y_train)
    print(station)
    break

print(model)


    


86
{86: LogisticRegression(max_iter=1500)}


In [61]:
print(test.summary())

AttributeError: 'LogisticRegression' object has no attribute 'summary'

In [40]:
df.columns

Index(['number', 'available_bikes', 'available_stands', 'dt', 'temperature',
       'main', 'day_of_week', 'hour'],
      dtype='object')